In [1]:
import mxnet as mx
from skimage import io
import numpy as np
from matplotlib import pyplot as plt
import time, os
import cPickle as cpkl
%matplotlib inline

print dir(mx.image)

['Augmenter', 'BrightnessJitterAug', 'CastAug', 'CenterCropAug', 'ColorJitterAug', 'ColorNormalizeAug', 'ContrastJitterAug', 'CreateAugmenter', 'CreateDetAugmenter', 'CreateMultiRandCropAugmenter', 'DetAugmenter', 'DetBorrowAug', 'DetHorizontalFlipAug', 'DetRandomCropAug', 'DetRandomPadAug', 'DetRandomSelectAug', 'ForceResizeAug', 'HorizontalFlipAug', 'HueJitterAug', 'ImageDetIter', 'ImageIter', 'LightingAug', 'RandomCropAug', 'RandomGrayAug', 'RandomOrderAug', 'RandomSizedCropAug', 'ResizeAug', 'SaturationJitterAug', 'SequentialAug', '__builtins__', '__doc__', '__file__', '__name__', '__package__', '__path__', 'absolute_import', 'center_crop', 'color_normalize', 'copyMakeBorder', 'cv2', 'det', 'detection', 'fixed_crop', 'image', 'imdecode', 'imread', 'imresize', 'io', 'json', 'logging', 'nd', 'np', 'numeric_types', 'os', 'print_function', 'random', 'random_crop', 'random_size_crop', 'recordio', 'resize_short', 'scale_down']


In [2]:
def remake_normal_rop(filter_file, data_root=''):
    normal, rop = [], []
    stage3_start = False
    stage2_start = False
    with open(os.path.join(filter_file), 'rd') as fl:
        for l in fl:
            l = l.strip().replace('\r','').replace('\n','')
            if '[stage2]' in l:
                stage2_start = True
                continue
            if '[stage3]' in l:
                stage3_start = True
                stage2_start = False
                continue
            if len(l.strip()) < 1:
                continue
            if stage3_start or stage2_start:
                sample = load_sample(l, data_root)
            if stage3_start:
                rop3.append(sample)
                continue
            if stage2_start:
                rop2.append(sample)
                continue
    syntext = [['stage2', 0], ['stage3', 1]]
    cpkl.dump(([[rop2, 0], [rop3, 1]], syntext), open(os.path.join(filter_file+'.pkl'),'wb'))
    return ([[rop2, 0], [rop3, 1]], syntext)

def remake_rop2or3(filter_file, data_root=''):
    rop2, rop3 = [], []
    stage3_start = False
    stage2_start = False
    with open(os.path.join(filter_file), 'rd') as fl:
        for l in fl:
            l = l.strip().replace('\r','').replace('\n','')
            if '[stage2]' in l:
                stage2_start = True
                continue
            if '[stage3]' in l:
                stage3_start = True
                stage2_start = False
                continue
            if len(l.strip()) < 1:
                continue
            if stage3_start or stage2_start:
                sample = load_sample(l, data_root)
            if stage3_start:
                rop3.append(sample)
                continue
            if stage2_start:
                rop2.append(sample)
                continue
    syntext = [['stage2', 0], ['stage3', 1]]
    cpkl.dump(([[rop2, 0], [rop3, 1]], syntext), open(os.path.join(filter_file+'.pkl'),'wb'))
    return ([[rop2, 0], [rop3, 1]], syntext)
def load_sample(sample_name, data_root=''):
    sample = []
    sample_name = os.path.join(data_root, sample_name)
    for img_name in os.listdir(sample_name):
        if img_name.split('.')[-1].lower() in ['jpg', 'png', 'jpeg']:
            sample.append(os.path.join(sample_name, img_name))
    if len(sample)< 1 or len(sample) > 15:
        print sample_name, len(sample)
    return sample

In [10]:
rop23, syntext = remake_rop2or3('/mnt/data/lab-data/medical/ROP_data_list/20170828/verified_data_list(stage2,3)_consistent.txt',
                                data_root='/mnt/data/lab-data/medical/ROP_data_all/')

/mnt/data/lab-data/medical/ROP_data_all/stage2_district2/f578e2fdea9fa381753bed8a0167c2a2_150417m4_20150417_OS 17
/mnt/data/lab-data/medical/ROP_data_all/stage2_district2/26455d9d7c518c0ae56aab14f3358fb8_zx0196692_20150622_OS 20
/mnt/data/lab-data/medical/ROP_data_all/stage2_district2/b0ab3c1b0fe872256db5aa619f529f95_zx0197704_20150619_OD 22
/mnt/data/lab-data/medical/ROP_data_all/stage2_district2/3f1d04c0cdff6fcc6520dc2b5ec498f5_150423m11_20150423_OD 17
/mnt/data/lab-data/medical/ROP_data_all/stage3_district2/e527e7a139d6d202b46d6b8c470f26ab_zx0203407_20150923_OS 21
/mnt/data/lab-data/medical/ROP_data_all/stage3_district2/67393095527394541598344cfba67a72_zx0213522_20151001_OS 19
/mnt/data/lab-data/medical/ROP_data_all/stage3_district2/eb0f5b251a98fe38245c60b0489aac96_zx0216228_20160107_OS 16
/mnt/data/lab-data/medical/ROP_data_all/stage3_district2/0e3d26c9d23ac37872fbad487900f290_zx0226388_20160126_OD 16
/mnt/data/lab-data/medical/ROP_data_all/stage3_district2/913ab618719e7ea3b0117aa4

In [12]:
len(rop23[0][0]),len(rop23[1][0])

(260, 260)

In [9]:
data_used_list_path = '/mnt/data/lab-data/medical/ROP_data_list/20170727/verified_data_list(normal,rop).txt'
flag = 'Whole'
start = False
normal = []
rop = []
with open(data_used_list_path) as fl:
    for l in fl:
        if len(l.strip())<1:
            continue
        if flag not in l and not start:
            continue
        else:
            start = True
        if 'Train' in l:
            start = False
            break
        if start and flag not in l:
            if 'normal' in l:
                normal.append(l.strip())
            else:
                rop.append(l.strip())

print 'normal:%d, rop:%d' % (len(normal), len(rop))
data_root='/mnt/data/lab-data/medical/ROP_data_all/'
normal = [load_sample(fn, data_root) for fn in normal]
rop = [load_sample(fn, data_root) for fn in rop]

syntext = [['normal', 0], ['rop', 1]]
cpkl.dump(([[normal, 0], [rop, 1]], syntext), open(os.path.join(data_used_list_path+'.pkl'),'wb'))

print normal

normal:1484, rop:742
/mnt/data/lab-data/medical/ROP_data_all/stage2_district2/f578e2fdea9fa381753bed8a0167c2a2_150417m4_20150417_OS 17
/mnt/data/lab-data/medical/ROP_data_all/stage2_district2/f578e2fdea9fa381753bed8a0167c2a2_150417m4_20150417_OD 16
/mnt/data/lab-data/medical/ROP_data_all/stage2_district2/b0ab3c1b0fe872256db5aa619f529f95_zx0197704_20150619_OD 22
/mnt/data/lab-data/medical/ROP_data_all/stage2_district2/3f1d04c0cdff6fcc6520dc2b5ec498f5_150423m11_20150423_OD 17
/mnt/data/lab-data/medical/ROP_data_all/stage2_district2/3487e94b9863e9782d68fda0e645ce46_zx0196268_20150625_OS 17
/mnt/data/lab-data/medical/ROP_data_all/stage2_district2/600d0dcba7ee381416bcc1f0bc54d9c9_150624m4_20150625_OD 22
/mnt/data/lab-data/medical/ROP_data_all/degenerative_stage_district2/600d0dcba7ee381416bcc1f0bc54d9c9_150625m4_20150625_OD 22
/mnt/data/lab-data/medical/ROP_data_all/stage2_district2/26455d9d7c518c0ae56aab14f3358fb8_zx0196692_20150622_OS 20
/mnt/data/lab-data/medical/ROP_data_all/stage5/eb52